In [2]:
"""
    Ideia principal desse projeto é entender mais a fundo os seguintes conceitos, descrevelos aqui e implementa-los:
        - Transformers:
            - Como funcionam
            - Como serem utilizados como modelo base para uma tarefa NLP
        - Embedding:
            - Como funcionam
            - Como utilizar para embedar minha base de treinamento especifica (Kaggle)
            - Ao predizer, como retornar para o estado de linguagem humana
        - Fine Tuning:
            - Realizar a construção de um modelo especifico para alguma tarefa de NLP
                - Limpeza dos dados
                - Pre processamento:
                    - Remoção de stop words
                    - Stemmizar os tokens
                - Realizar uma pipeline:
                    - Conversão para embedding
                    - Construção do modelo para a task
        - Exibição para o usuário final:
            - Exibir graficos
                - Comparação entre metricas:
                    - F1 Score
                    - Recall
"""

'\n    Ideia principal desse projeto é entender mais a fundo os seguintes conceitos, descrevelos aqui e implementa-los:\n        - Transformers:\n            - Como funcionam\n            - Como serem utilizados como modelo base para uma tarefa NLP\n        - Embedding:\n            - Como funcionam\n            - Como utilizar para embedar minha base de treinamento especifica (Kaggle)\n            - Ao predizer, como retornar para o estado de linguagem humana\n        - Fine Tuning:\n            - Realizar a construção de um modelo especifico para alguma tarefa de NLP\n                - Limpeza dos dados\n                - Pre processamento:\n                    - Remoção de stop words\n                    - Stemmizar os tokens\n                - Realizar uma pipeline:\n                    - Conversão para embedding\n                    - Construção do modelo para a task\n        - Exibição para o usuário final:\n            - Exibir graficos\n                - Comparação entre metric

### Transformers

### Embedding

In [3]:
"""
    Para esse caso nós poderiamos utilizar muitas maneiras de vetorizar as palavras para realizar
    o fine tuning de nosso modelo, como por exemplo TFID Vectorizer, Word2Vec, entre outras.
    Mas por uma escolha mais precisa optamos por utilizar o BERT para embedar nossas palavras.
    E por que o BERT e não alguma dessas outras?
        - Em comparação entre BERT e Word2Vec, o BERT se sai melhor por conta de embedar usando o contexto
        da frase por completo, e não só gerar um embed estatico igual o Word2Vec, além de ter seu pré treino em
        uma base de dados muito maior ao do Word2Vec. E com isso BERT se torna um modelo melhor para tarefas de NLP.

"""

'\n    Para esse caso nós poderiamos utilizar muitas maneiras de vetorizar as palavras para realizar\n    o fine tuning de nosso modelo, como por exemplo TFID Vectorizer, Word2Vec, entre outras.\n    Mas por uma escolha mais precisa optamos por utilizar o BERT para embedar nossas palavras.\n    E por que o BERT e não alguma dessas outras?\n        - Em comparação entre BERT e Word2Vec, o BERT se sai melhor por conta de embedar usando o contexto\n        da frase por completo, e não só gerar um embed estatico igual o Word2Vec, além de ter seu pré treino em\n        uma base de dados muito maior ao do Word2Vec. E com isso BERT se torna um modelo melhor para tarefas de NLP.\n\n'

### Fine Tuning

### Construção

In [23]:
import nltk
import re

import pandas as pd
import numpy as np
import tensorflow as tf

from string import punctuation
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from transformers import TFBertForSequenceClassification

In [5]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

In [6]:
nltk.download('stopwords')
STOPWORDS = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\janna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df = pd.read_csv("./dataset/data.csv")
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [8]:
df.shape

(5842, 2)

In [9]:
df["Sentiment"] = np.where(df["Sentiment"] == "positive", 1, 0)

In [10]:
def pre_process_text(text):
    """
    This function is used to tokenize each phrase, remove punctuation and stopwords,
    after that stem each word and join each token to an string in lower case.
    """
    text = [stemmer.stem(word) for word in text.split() if word not in punctuation and word not in STOPWORDS]
    return " ".join(text).lower()

In [11]:
df["Sentence"] = df["Sentence"].apply(pre_process_text)

In [12]:
df.head()

,Sentence,Sentiment
0,the geosolut technolog leverag benefon 's gps ...,1
1,"$esi lows, $1.50 $2.50 bk real possibl",0
2,for last quarter 2010 componenta 's net sale d...,1
3,accord finnish-russian chamber commerc major c...,0
4,the swedish buyout firm sold remain 22.4 perce...,0


In [13]:
X = df["Sentence"]
y = df[["Sentiment"]]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [15]:
X_train.shape

(4673,)

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [17]:
X_train_embed = tokenizer( 
    text=X_train.tolist(),
    padding="max_length",
    truncation=True,
    return_tensors='tf',
    add_special_tokens=True
)

In [18]:
X_train_embed['input_ids']

<tf.Tensor: shape=(4673, 512), dtype=int32, numpy=
array([[  101,  2918,  2640, ...,     0,     0,     0],
       [  101, 15253,  2080, ...,     0,     0,     0],
       [  101, 15802, 12478, ...,     0,     0,     0],
       ...,
       [  101,  3389,  2094, ...,     0,     0,     0],
       [  101,  6887, 27292, ...,     0,     0,     0],
       [  101, 11090,  2368, ...,     0,     0,     0]])>

In [19]:
X_test_embed = tokenizer( 
    text=X_test.tolist(),
    padding="max_length",
    truncation=True,
    return_tensors='tf',
    add_special_tokens=True
)

In [20]:
def encode_dataset(encodings, labels):
    return tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    ))

train_dataset = encode_dataset(X_train_embed, y_train)
test_dataset = encode_dataset(X_test_embed, y_test)

In [24]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model.fit(train_dataset.shuffle(100).batch(8), epochs=2, batch_size=8)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
585/585 [==============================] - 3972s 7s/step - loss: 0.6332 - accuracy: 0.6824
Epoch 2/2
585/585 [==============================] - 3845s 7s/step - loss: 0.6298 - accuracy: 0.6839


In [ ]:
"""
    Fontes utilizadas para pesquisa:
        - https://www.geeksforgeeks.org/how-to-generate-word-embedding-using-bert/

"""